# Trabalho Prático 1
## Algoritmos Geométricos
**Proposta:** Envoltória Convexa como algoritmo de classificação  
**Desenvolvedores:**
- Pedro de Oliveira Guedes
- Gabriel Bifano Fredi
- Tarcísio Augusto Santos Lafaiete

# Introdução
Este trabalho prático tem como motivação a aplicação prática dos algoritmos geométricos estudados durante as aulas de Algoritmos II, buscando a fixação do conhecimento. Para obter essa fixação, o trabalho prático propõe a utilização dos algoritmos estudados para a construção de um classificador de conjuntos de dados bidimensionais.  
  
Para que o objetivo proposto por este trabalho seja alcançado, será desenvolvida uma série de algoritmos com funções distintas que, quando combinados, tornarão a classificação de pontos em um espaço bidimensional possível, apesar de pouco precisa. Devem ser desenvolvidos algoritmos para os seguintes propósitos:
1. Encontrar a envoltória convexa de um conjunto de pontos;
2. Determinar se duas envoltórias convexas se cruzam ou se sobrepõem;
3. Encontrar uma equação linear que separe duas envoltórias através dos pontos mais próximos entre elas;
4. Classificar um conjunto de pontos de acordo com a equação de separabilidade linear obtida.
  
Após a execução de todos os algoritmos criados para satisfazer os propósitos anteriormente apresentados, a classificação obtida será avaliada de acordo com as métricas de: "*precisão*", "*revocação*" e "*f1-escore*". Para a avaliação dos algoritmos criados, serão utilizados 10 *datasets*, divididos entre 3 *datasets* gerados de forma aleatória e 7 *datasets* do mundo real.  

Ao fim da execução deste trabalho, espera-se que o entendimento de algoritmos geométricos obtidos ao longo da disciplina sejam amplificados, tornando o aprendizado mais valioso academicamente e para o mercado de trabalho.

# Implementação
Ao longo desta seção, serão apresentadas as implementações feitas para os propósitos apresentados na introdução, juntamente com uma análise do porquê determinada implementação foi escolhida em detrimento de outra, o funcionamento e a complexidade de espaço e tempo da mesma.

## Bibliotecas utilizadas
As bibliotecas utilizadas para o desenvolvimento do presente projeto serão declaradas na próxima célula de código. Entre elas, estão bibliotecas para aplicação de algoritmos de álgebra linear, facilitação de cálculos matemáticos, melhora na visualização de dados, entre outras. Elas serão brevemente discutidas, apresentando o propósito de utilização, a seguir.
  
### Display
Essa biblioteca tem por objetivo melhorar a visualização dos dados tabelados, fazendo com que eles apareçam formatados na tela, o que nem sempre acontece ao realizar um simples "*print()*"  
  
### Math
Essa é a biblioteca padrão do python para operações mais sofisticadas de matemática, como a utilização da função de raiz quadrada "*sqrt()*".  

### Matplotlib
A biblioteca *matplotlib* capacita o programa a fornecer feedback visual dos pontos, funções e envoltórias encontradas ao longo da execução, permitindo uma forte intuição do comportamento do algoritmo, e facilitando o processo de *debug*.
  
### Numpy
A biblioteca *numpy* fornece uma série de facilidades para o manejo e a transformação de vetores, que são usados extensamente no algoritmo.
  
### Pandas
A biblioteca *pandas* é uma escolha padrão para trabalhar com ciência de dados, sendo utilizada ao longo deste projeto para agrupar e tratar os conjuntos de dados na aplicação da classificação sobre eles.  
  
### Sklearn
A biblioteca *sklearn* foi utilizada neste projeto para calcular as métricas de *precisão*, *revocação* e *f1-escore*, além da utilização da função generativa de *datasets* e a importação facilitada de *datasets* reais.  

In [22]:
from IPython.display import display
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics as sk_metrics
from sklearn import datasets as sk_datasets

## Envoltória convexa
O algoritmo escolhido para realizar esta tarefa é o da **Varredura de Graham**, que possui complexidade de tempo *O(n*log n)* e de espaço *O(n)*. Ele foi escolhido em detrimento do algoritmo **Gift Wrapping** proposto por Jarvis, devido à menor complexidade de tempo no pior caso. Como a distribuição dos pontos após a redução de dimensionalidade dos dados é imprevisível, não é possível afirmar para todos os casos que a quantidade de vértices será mínima.
  
Esse algoritmo funciona fazendo a ordenação dos pontos no espaço bidimensional em relação a um vértice denominado âncora (*O(n log n)*), que é escolhido como sendo o que possui menor coordenada Y, utilizando a menor coordenada X como critério de desempate.  
  
Após o encontro do âncora e a ordenação dos pontos, o algoritmo itera por cada um dos vértices verificando se eles devem ou não ser incluídos como vértices da envoltória (*O(n)*), ou se eles já fazem parte como ponto interno da mesma. Para verificar se um ponto *p3* será ou não incluído como vértice do polígono, é averiguada a direção do giro feito pelo segmento *p2 -> p3* em relação a *p1 -> p2*.  
  
Caso o giro à esquerda, o ponto p3 é incluído como vértice da envoltória. Caso contrário, o ponto *p2* é descartado e a mesma verificação é feita para o segmento *p1 -> p3* em relação a *p0 -> p1*.

### Critério de comparação
Para realizar a comparação entre dois pontos distintos, foi criada a estrutura de dados denominada *Vertex*, que armazena o vértice atual e o âncora a partir de qual todos os vértices foram normalizados. A partir dessa definição, o método de comparaçõ "*<*" foi sobrescrito, tornando possível a comparação entre dois membros da mesma classe "*Vertex*" e, consequentemente, a ordenação desses elementos.  

A decisão de um vértice ser menor do que outro é feita de acordo com a função "*turns_left(p0, p1, p2)*", que verifica através de álgebra linear se o segmento formado por *p1 -> p2* faz um giro à esquerda em relação a *p0 -> p1*, retornando verdadeiro, caso positivo.

In [23]:
def turns_left(p0: tuple, p1: tuple, p2: tuple):
    matrix = np.array([
            [(p1[0] - p0[0]), (p1[1] - p0[1])],
            [(p2[0] - p0[0]), (p2[1] - p0[1])]
        ])
    return np.linalg.det(matrix) < 0.0

class Vertex:
    def __init__(self, anchor: tuple, point: tuple):
        self.anchor = anchor
        self.point = point

    def __repr__(self):
        return f'({self.point[0]}, {self.point[1]})'

    def __lt__(self, other):
        return turns_left(p0 = self.anchor, p1 = self.point, p2 = other.point)

### Encontrando o âncora
Para encontrar o vértice âncora do conjunto, o algoritmo assumirá que o primeiro ponto é o âncora inicialmente. Após isso, será iterado pelos outros vértices do conjunto, verificando se o atual âncora está acima do atual vértice comparado.  
  
Caso ele esteja, o atual vértice vira o novo âncora. Caso ambos estejam em uma mesma coordenada Y, então o que possuir a menor coordenada X será considerado âncora.  
  
Dessa forma, o seguinte algoritmo possui complexidade de tempo *O(n)*, por iterar pelos *n* vértices disponíveis e espaço *O(n)* já que não cria nenhuma estrutura auxiliar para a busca.

In [25]:
def find_anchor(point_set: list[tuple]) -> tuple:
    current_anchor = point_set[0]

    for curr_point in point_set[1:]:
        if curr_point[1] < current_anchor[1]: # Comparando Y
            current_anchor = curr_point
        if curr_point[1] == current_anchor[1]:
            current_anchor = curr_point if curr_point[0] < current_anchor[0] else current_anchor # Comparando X no caso de empate

    return current_anchor

### Ordenando os segmentos
A ordenação dos segmentos será baseada no critério de comparação definido anteriormente.  

Para isso, o âncora será removido do conjunto inicialmente e readicionado ao fim. Para que a ordenação ocorra de forma facilitada, os elementos do conjunto serão transformados no tipo **Vertix** definido acima.

In [4]:
def sort_points_set(anchor: tuple, points_set: list[tuple]) -> list[tuple]:
    points_set.remove(anchor)

    vertix_set = [Vertix(anchor, (point[0], point[1])) for point in points_set]

    vertix_set.sort()

    sorted_points = [vertix.point for vertix in vertix_set]

    return sorted_points.insert(0, anchor)

### Algoritmo final
Após as definições anteriores, o algoritmo final pode ser montado de acordo com o funcionamento que foi descrito no texto da seção **Envoltória Convexa**.

In [5]:
def graham_scan(points_set: list[tuple]) -> list[tuple]:
    anchor = find_anchor(point_set = points_set)
    
    sorted_points = sort_points_set(anchor = anchor, points_set = points_set)

    convex_hull_vertices = [point for point in sorted_points[:3]]

    for current_point in sorted_points[3:]:
        top = convex_hull_vertices.pop()
        next_to_top = convex_hull_vertices.pop()
        
        while not turns_left_or_keeps_direction(p0 = next_to_top, p1 = top, p2 = current_point):
            top = next_to_top
            next_to_top = convex_hull_vertices.pop()

        convex_hull_vertices.append(next_to_top)
        convex_hull_vertices.append(top)
        convex_hull_vertices.append(current_point)

    return convex_hull_vertices

## Verificação de separabilidade linear
Somente encontrar a envoltória convexa para as diferentes classes dos dados não é suficiente, visto que, eventualmente, essas classes podem se sobrepor no espaço bidimensional, comprometendo a classificação posterior dos pontos.  

Dessa forma, será empregado o algoritmo de **varredura linear** para verificar se há interseção entre os segmentos das envoltórias convexas. Caso haja essa interseção, isso significa que não há separação linear para os dados estudados. Caso contrário, os dados podem ser separados linearmente.  

Para que a varredura linear funcione corretamente, será necessário definir algumas estruturas de dados e funções auxiliares, como:
1. **Mapas de origem e destino:** Essas estruturas facilitarão na identificação dos vértices que são origem de determinado segmento e os que são o destino, atribuindo o estado de "*origem*" àqueles vértices com menor coordenada X.
2. **Vértices da envoltória:** Essa estrutura armazena as coordenadas dos vértices da envoltória, definindo o método de comparação baseado na coordenada X, para facilitar o processo de ordenação.
3. **Ordenação dos pontos da envoltória:** Essa será uma função auxiliar para o processo de ordenação, removendo esta responsabilidade do algoritmo final.
4. **Segmentos de reta:** Como a ordenação da árvore binária balanceada é baseada na coordenada Y em que a reta de varredura intercepta o segmento, será criada uma estrutura específica para esta aplicação, definindo corretamente o método de comparação.
5. **Árvore binária balanceada:** Essa estrutura é necessária para o funcionamento correto da varredura linear, já que ela é utilizada para conferir quando os segmentos estão acima ou abaixo uns dos outros.
6. **Verificação de interseção entre duas retas:** Essa função com complexidade de tempo *O(1)* será a responsável por verificar as interseções entre os pares de segmentos evidenciados durante a varredura linear.

### Mapas de origem e destino
A função iterará pelos vértices da envoltória convexa, analisando os vértices par a par e atribuindo-os aos dicionários de origem e destino. Isso facilitará a identificação dos mesmos durante a varredura.

In [6]:
def get_destines_and_origins_maps(vertices: list[tuple]) -> tuple:
    destines = {vertix: [] for vertix in vertices}
    origins = {vertix: [] for vertix in vertices}
    for index in range(0, len(vertices)):
        a = vertices[index]
        b = vertices[(index + 1) % len(vertices)]
        if a[0] < b[0]:
            destines[a].append(b)
            origins[b].append(a)
        else:
            destines[b].append(a)
            origins[a].append(b)

    return (destines, origins)

#### Testes
Testando função implementada.

In [7]:
test = [(1, 1), (2, 2), (1, 3), (0, 2)]
d, o = get_destines_and_origins_maps(test)
print(f'Origens: {o}')
print(f'Destinos: {d}')

Origens: {(1, 1): [(0, 2)], (2, 2): [(1, 1), (1, 3)], (1, 3): [(0, 2)], (0, 2): []}
Destinos: {(1, 1): [(2, 2)], (2, 2): [], (1, 3): [(2, 2)], (0, 2): [(1, 3), (1, 1)]}


### Vértices da envoltória
Essa estrutura armazena a coordenada X e Y do ponto, comparando-o com outros pontos de acordo com a coordenada X da origem do mesmo, quebrando empates pela coordenada X do ponto de destino e quebrando um último empate posicionando os pontos com coordenadas Y menores primeiro.

In [8]:
class Vertix:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'[{self.is_destine}]({self.x}, {self.y})'

    def __lt__(self, other):
        if self.x < other.x:
            return True
        if self.x == other.x:
            return self.y < other.y
        return False

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

### Ordenação dos pontos da envoltória
Essa função receberá a lista de pontos, realizará a conversão para a estrutura criada para armazenamento dos vérties, os ordenará e devolverá os pontos no mesmo formato em que foram recebidos.

In [9]:
def sort_vertices(vertices: list[tuple]) -> list[tuple]:
    aux_list = []
    for index in range(0, len(vertices)):
        a = vertices[index]
        b = vertices[(index + 1) % len(vertices)]
        aux_list.append( Vertix(a[0], a[1]) )
    aux_list.sort()

    final_list = []
    for vertix in aux_list:
        final_list.append( (vertix.x, vertix.y) )
    return final_list

#### Testes
Verificando o funcionamento da função implementada.

In [10]:
test = [(1, 1), (2, 2), (1, 3), (0, 2)]

print(sort_vertices(test))

[(0, 2), (1, 1), (1, 3), (2, 2)]


### Nós da árvore binária balanceada
Para essa estrutura, será necessário armazenar quatro elementos principais, que facilitarão as computações seguintes:
- Identificador da envoltória
    - Como os vértices de uma mesma envoltória se tocam, eles não podem ser considerados como interseção. Esse identificador ajuda a prevenir o problema.
- Ponto de origem do segmento
- Ponto de fim do segmento
- Coeficiente angular da equação da reta
- Constante da equação da reta

Note que os segmentos serão ordenados na árvore binária de acordo o valor de Y em que a reta de varredura intercepta o dado segmento. Tal valor é calculado a partir da coordenada X recebida, que é a posição atual da reta de varredura.

In [11]:
class SegmentNode:
    RED = True
    BLACK = False

    def __init__(self, origin: tuple, destine: tuple, hull_id: int, color = RED, alpha = 0.01):
        self.color = color
        self.left = self.right = self.parent = NullSegmentNode.instance()

        self.hull_id = hull_id
        self.origin = origin
        # É provocada uma pequena perturbação na coordenada X do destino caso ela seja igual à da origem, evitando segmentos puramente verticais
        self.destine = destine if destine[0] != origin[0] else (destine[0] + alpha, destine[1])

        self.slope = (destine[1] - origin[1])/(destine[0] - origin[0]) # Coeficiente angular (m)
        self.constant = destine[1] - self.slope * destine[0] # Constante da reta (b = y - mx)

    def __str__(self, level = 0, indent = "   "):
        s = level * indent + str(self.origin) + " --> " + str(self.destine)
        if self.left:
          s = s + "\n" + self.left.__str__(level + 1, indent)
        if self.right:
          s = s + "\n" + self.right.__str__(level + 1, indent)
        return s

    def get_y_intercetp_coord(self, sweep_x: float) -> float:
        return self.slope * sweep_x + self.constant

    def __nonzero__(self):
        return True

    def __bool__(self):
        return True



class NullSegmentNode(SegmentNode):
    __instance__ = None

    @classmethod
    def instance(self):
        if self.__instance__ is None:
            self.__instance__ = NullSegmentNode()
        return self.__instance__

    def __init__(self):
        self.color = SegmentNode.BLACK
        self.hull_id = self.origin = self.destine = self.slope = self.constant = None
        self.left = self.right = self.parent = None

    def __nonzero__(self):
        return False

    def __bool__(self):
        return False

### Árvore binária balanceada
**Referência:** https://github.com/headius/redblack/tree/master

In [12]:
class RedBlackTree:
    def __init__(self):
        self.root = NullSegmentNode.instance()
        self.size = 0

    def __str__(self):
        return str(self.root)

    def add(self, origin: tuple, destine: tuple, hull_id: int):
        self.insert(SegmentNode(origin, destine, hull_id))

    def insert(self, node: SegmentNode):
        self.__insert_helper(node)

        node.color = SegmentNode.RED
        while node != self.root and node.parent.color == SegmentNode.RED:
            if node.parent == node.parent.parent.left: # Verifica se o pai do nó atual está à esquerda do avô
                aux = node.parent.parent.right
                if aux and aux.color == SegmentNode.RED:
                    node.parent.color = SegmentNode.BLACK
                    aux.color = SegmentNode.BLACK
                    node.parent.parent.color = SegmentNode.RED
                    node = node.parent.parent
                else:
                    if node == node.parent.right:
                        node = node.parent
                        self.__left_rotate(node)
                    node.parent.color = SegmentNode.BLACK
                    node.parent.parent.color = SegmentNode.RED
                    self.__right_rotate(node.parent.parent)
            else:
                aux = node.parent.parent.left
                if aux and aux.color == SegmentNode.RED:
                    node.parent.color = SegmentNode.BLACK
                    aux.color = SegmentNode.BLACK
                    node.parent.parent.color = SegmentNode.RED
                    node = node.parent.parent
                else:
                    if node == node.parent.left:
                        node = node.parent
                        self.__right_rotate(node)
                    node.parent.color = SegmentNode.BLACK
                    node.parent.parent.color = SegmentNode.RED
                    self.__left_rotate(node.parent.parent)
        self.root.color = SegmentNode.BLACK


    def __insert_helper(self, node: SegmentNode):
        sweep_x = node.origin[0]

        aux = NullSegmentNode.instance()
        tree_root = self.root
        while tree_root: # Buscando um nó à esquerda ou à direita que esteja vazio para posicionar o novo elemento
          aux = tree_root
          if node.get_y_intercetp_coord(sweep_x) < tree_root.get_y_intercetp_coord(sweep_x):
            tree_root = tree_root.left # Elemento será inserido à esquerda da raiz anterior
          else:
            tree_root = tree_root.right # Elemento será inserido à direita da raiz anterior

        node.parent = aux # Definindo o pai do nó a ser inserido
        if not aux:
          self.root = node # Caso em que a árvore está vazia
        else:
          if node.get_y_intercetp_coord(sweep_x) < aux.get_y_intercetp_coord(sweep_x):
            aux.left = node # Nó a ser inserido está à esquerda do pai
          else:
            aux.right = node # Nó a ser inserido está à direita do pai

        self.size += 1

    def __left_rotate(self, node: SegmentNode):
        if not node.right:
            raise "node.right is null!"
        aux = node.right
        node.right = aux.left
        if aux.left:
            aux.left.parent = node
        aux.parent = node.parent
        if not node.parent:
            self.root = aux
        else:
            if node == node.parent.left:
                node.parent.left = aux
            else:
                node.parent.right = aux
        aux.left = node
        node.parent = aux

    def __right_rotate(self, node: SegmentNode):
        if not node.left:
            raise "node.left is null!"
        aux = node.left
        node.left = aux.right
        if aux.right:
            aux.right.parent = node
        aux.parent = node.parent
        if not node.parent:
            self.root = aux
        else:
            if node == node.parent.left:
                node.parent.left = aux
            else:
                node.parent.right = aux
        aux.right = node
        node.parent = aux

    def search_segment(self, sweep_x: float, target_y: float, origin: tuple, destine: tuple) -> SegmentNode:
        root = self.root
        while root:
            if root.get_y_intercetp_coord(sweep_x) != target_y:
                if root.get_y_intercetp_coord(sweep_x) < target_y:
                    root = root.right
                else:
                    root = root.left
            else:
                if root.origin != origin or root.destine != destine:
                    if root.left and root.left.get_y_intercetp_coord(sweep_x) == target_y:
                        return root.left
                    elif root.right and root.right.get_y_intercetp_coord(sweep_x) == target_y:
                        return root.right
                    else:
                        return NullSegmentNode.instance()
                else:
                    return root
        return root

    def remove(self, node: SegmentNode):
        if not node:
            return
        if not node.left or not node.right:
            aux1 = node
        else:
            aux1 = self.successor(node)

        if not aux1.left:
            aux2 = aux1.right
        else:
            aux2 = aux1.left
        aux2.parent = aux1.parent

        if not aux1.parent:
            self.root = aux2
        else:
            if aux1 == aux1.parent.left:
                aux1.parent.left = aux2
            else:
                aux1.parent.right = aux2

        if aux1 != node:
            node.origin = aux1.origin
            node.destine = aux1.destine
            node.slope = aux1.slope
            node.constant = aux1.constant

        if aux1.color == SegmentNode.BLACK:
            self.__delete_fixup(aux2)

        self.size -= 1
        return aux1

    def successor(self, node: SegmentNode):
        if node.right:
            return self.minimum(node.right)
        aux = node.parent
        while aux and node == aux.right:
            node = aux
            aux = aux.parent
        return aux

    def minimum(self, node: SegmentNode = None):
        if not node:
            node = self.root
        while node.left:
            node = node.left
        return node

    def predecesor(self, node: SegmentNode):
        if node.left:
            return self.maximum(node.left)
        aux = node.parent
        while aux and node == aux.left:
            node = aux
            aux = aux.parent
        return aux

    def maximum(self, node: SegmentNode = None):
        if not node:
            node = self.root
        while node.right:
            node = node.right
        return node

    def __delete_fixup(self, node: SegmentNode):
        while node != self.root and node.color == SegmentNode.BLACK:
            if node == node.parent.left:
                aux = node.parent.right
                if aux.color == SegmentNode.RED:
                    aux.color = SegmentNode.BLACK
                    node.parent.color = SegmentNode.RED
                    self.__left_rotate(node.parent)
                    aux = node.parent.right
                if aux.left.color == SegmentNode.BLACK and aux.right.color == SegmentNode.BLACK:
                    aux.color = SegmentNode.RED
                    node = node.parent
                else:
                    if aux.right.color == SegmentNode.BLACK:
                        aux.left.color = SegmentNode.BLACK
                        aux.color = SegmentNode.RED
                        self.__right_rotate(aux)
                        aux = node.parent.right
                    aux.color = node.parent.color
                    node.parent.color = SegmentNode.BLACK
                    aux.right.color = SegmentNode.BLACK
                    self.__left_rotate(node.parent)
                    node = self.root
            else:
                aux = node.parent.left
                if aux.color == SegmentNode.RED:
                    aux.color = SegmentNode.BLACK
                    node.parent.color = SegmentNode.RED
                    self.__right_rotate(node.parent)
                    aux = node.parent.left
                if aux.right.color == SegmentNode.BLACK and aux.left.color == SegmentNode.BLACK:
                    aux.color = SegmentNode.RED
                    node = node.parent
                else:
                    if aux.left.color == SegmentNode.BLACK:
                        aux.right.color = SegmentNode.BLACK
                        aux.color = SegmentNode.RED
                        self.__left_rotate(aux)
                        aux = node.parent.left
                    aux.color = node.parent.color
                    node.parent.color = SegmentNode.BLACK
                    aux.left.color = SegmentNode.BLACK
                    self.__right_rotate(node.parent)
                    node = self.root

        node.color = SegmentNode.BLACK

#### Testando a árvore binária balanceada
Ao longo do desenvolvimento da árvore binária balanceada, foram realizados alguns testes para verificar sua corretude. Os testes e seus resultados podem ser encontrados na célula de código abaixo.

In [17]:
test = None
print("====== Impressão inicial =====")
print(test)
print("\n\n\n\n")

test = RedBlackTree()
test.add((0, 2), (1, 1), 1) # d
test.add((0, 2), (1, 3), 1) # c
test.add((1, 1), (2, 2), 1) # a
test.add((1, 3), (2, 2), 1) # b

print("====== Árvore completa =====")
print(test)
print("\n\n\n\n")
print("====== Segmento '(1, 1) --> (2, 2)' que é a raiz da árvore =====")
print(test.search_segment(sweep_x=1, target_y=1, origin=(1, 1), destine=(2, 2)))
print("\n\n\n\n")
print("====== Antecessor do Segmento '(1, 1) --> (2, 2)' =====")
print(test.predecesor( test.search_segment(sweep_x=1, target_y=1, origin=(1,1), destine=(2,2)) ))
print("\n\n\n\n")
print("====== Segmento '(0, 2) --> (1, 1)' =====")
print( test.search_segment(sweep_x=0, target_y=2, origin=(0,2), destine=(1,1)) )
print("\n\n\n\n")
print("====== Antecessor do Segmento '(0, 2) --> (1, 1)' =====")
print(test.predecesor( test.search_segment(sweep_x=0, target_y=2, origin=(0,2), destine=(1,1)) ))
print("\n\n\n\n")
print("====== Sucessor do Segmento '(0, 2) --> (1, 1)' =====")
print(test.successor( test.search_segment(sweep_x=0, target_y=2, origin=(0,2), destine=(1,1)) ))
print("\n\n\n\n")

test.remove(test.predecesor( test.search_segment(sweep_x=1, target_y=1, origin=(1,1), destine=(2,2)) ))
test.remove(NullSegmentNode.instance())
print("====== Árvore completa após a remoção do antecessor da raiz '(0, 2) --> (1, 1)' =====")
print(test)

====== Impressão inicial =====
None





====== Árvore completa =====
(1, 1) --> (2, 2)
   (0, 2) --> (1, 1)
   (0, 2) --> (1, 3)
      (1, 3) --> (2, 2)





====== Segmento '(1, 1) --> (2, 2)' que é a raiz da árvore =====
(1, 1) --> (2, 2)
   (0, 2) --> (1, 1)
   (0, 2) --> (1, 3)
      (1, 3) --> (2, 2)





====== Antecessor do Segmento '(1, 1) --> (2, 2)' =====
(0, 2) --> (1, 1)





====== Segmento '(0, 2) --> (1, 1)' =====
None --> None





====== Antecessor do Segmento '(0, 2) --> (1, 1)' =====
None --> None





====== Sucessor do Segmento '(0, 2) --> (1, 1)' =====
(0, 2) --> (1, 3)
   (1, 1) --> (2, 2)
   (1, 3) --> (2, 2)





====== Árvore completa após a remoção do antecessor da raiz '(0, 2) --> (1, 1)' =====
(0, 2) --> (1, 3)
   (1, 1) --> (2, 2)
   (1, 3) --> (2, 2)


### Verificação de interseção entre duas retas
Blá blá blá

In [14]:
def direction(p0: tuple, p1: tuple, p2: tuple) -> float:
    matrix = np.array([
            [(p1[0] - p0[0]), (p1[1] - p0[1])],
            [(p2[0] - p0[0]), (p2[1] - p0[1])]
        ])
    return np.linalg.det(matrix)

def in_segment_bounds(origin: tuple, destine: tuple, target: tuple) -> bool:
    in_x_bounds = (target[0] >= origin[0] and target[0] <= destine[0])
    in_y_bounds = (target[1] >= origin[1] and target[1] <= destine[1])

    return in_x_bounds and in_y_bounds

def segments_intersect(s1: SegmentNode, s2: SegmentNode) -> bool:
    if not s1 or not s2:
        return False
    
    if s1.hull_id == s2.hull_id:
        return False
    
    d1 = direction(s1.origin, s1.destine, s2.origin)
    d2 = direction(s1.origin, s1.destine, s2.destine)
    d3 = direction(s2.origin, s2.destine, s1.origin)
    d4 = direction(s2.origin, s2.destine, s1.destine)

    if ( (d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0) ) and ( (d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0) ):
        return True
    elif d1 == 0 and in_segment_bounds(s1.origin, s1.destine, s2.origin):
        return True
    elif d2 == 0 and in_segment_bounds(s1.origin, s1.destine, s2.destine):
        return True
    elif d3 == 0 and in_segment_bounds(s2.origin, s2.destine, s1.origin):
        return True
    elif d4 == 0 and in_segment_bounds(s2.origin, s2.destine, s1.destine):
        return True
    else:
        return False

### Varredura linear
Blá blá blá

In [15]:
def linear_sweeping(first_convex_hull: list[tuple], second_convex_hull: list[tuple]) -> bool:
    destines1, origins1 = get_destines_and_origins_maps(first_convex_hull)
    destines2, origins2 = get_destines_and_origins_maps(second_convex_hull)

    sorted_vertices = sort_vertices(first_convex_hull.extend(second_convex_hull)) # Todos os vértices

    rbt = RedBlackTree()

    for vertix in sorted_vertices:
        destines = destines1.get(vertix)
        if destines and len(destines) > 0: # Existe um ou mais vértices de destino partindo do atual
            if destines2.get(vertix) or origins2.get(vertix): # Coordenadas de um vértice são compartilhadas entre as envoltórias
                return True

            for destine in destines:
                rbt.add(vertix, destine, 1)
                curr_segment = rbt.search_segment(vertix[0], vertix[1], vertix, destine)
                curr_successor = rbt.successor(curr_segment)
                curr_predecessor = rbt.predecesor(curr_segment)
                if segments_intersect(curr_segment, curr_successor) or segments_intersect(curr_segment, curr_predecessor):
                    return True
        destines = destines2.get(vertix)
        
        if destines and len(destines) > 0:
            if destines1.get(vertix) or origins1.get(vertix): # Coordenadas de um vértice são compartilhadas entre as envoltórias
                return True

            for destine in destines:
                rbt.add(vertix, destine, 2)
                curr_segment = rbt.search_segment(vertix[0], vertix[1], vertix, destine)
                curr_successor = rbt.successor(curr_segment)
                curr_predecessor = rbt.predecesor(curr_segment)
                if segments_intersect(curr_segment, curr_successor) or segments_intersect(curr_segment, curr_predecessor):
                    return True

        
        origins = origins1.get(vertix)
        if origins and len(origins) > 0: # Existe um ou mais vértices de origem que levam ao atual
            if origins2.get(vertix) or destines2.get(vertix): # Coordenadas de um vértice são compartilhadas entre as envoltórias
                return True

            for origin in origins:
                curr_segment = rbt.search_segment(vertix[0], vertix[1], origin, vertix)
                curr_successor = rbt.successor(curr_segment)
                curr_predecessor = rbt.predecesor(curr_segment)
                if segments_intersect(curr_predecessor, curr_successor):
                    return True
                rbt.remove(curr_segment)
        origins = origins2.get(vertix)
        
        if origins and len(origins) > 0:
            if origins1.get(vertix) or destines1.get(vertix): # Coordenadas de um vértice são compartilhadas entre as envoltórias
                return True

            for origin in origins:
                curr_segment = rbt.search_segment(vertix[0], vertix[1], origin, vertix)
                curr_successor = rbt.successor(curr_segment)
                curr_predecessor = rbt.predecesor(curr_segment)
                if segments_intersect(curr_predecessor, curr_successor):
                    return True
                rbt.remove(curr_segment)
    return False

### Definição da separabilidade linear
Os conjuntos de dados serão separaveis linearmente, sempre que as envoltórias convexas não possuirem nenhuma interseção na verificação por varredura linear.

In [16]:
def linear_separability(convex_hull_1: list[tuple], convex_hull_2: list[tuple]) -> bool:
    return not linear_sweeping(convex_hull_1, convex_hull_2)

## Construção do modelo
Blá blá blá

## Classificador
Blá blá blá

## Representação gráfica da classificação
Blá blá blá

In [ ]:
class GraphicRepresentation:
    def __init__(self):
        self.first_label_color = "blue"
        self.second_label_color = "orange"
        self.intersection_color = "red"
        self.connection_color = "gray"
        self.model_line_color = "green"

    def plot_train_points(self, first_label_points: list[tuple], second_label_points: list[tuple]):
        self.first_label_train_x = [point[0] for point in first_label_points]
        self.first_label_train_y = [point[1] for point in first_label_points]
        plt.scatter(self.first_label_train_x, self.first_label_train_y, color=self.first_label_color)

        self.second_label_train_x = [point[0] for point in second_label_points]
        self.second_label_train_y = [point[1] for point in second_label_points]
        plt.scatter(self.second_label_train_x, self.second_label_train_y, color=self.second_label_color)

        plt.show()

    def plot_convex_hull(self, first_convex_hull: list[tuple], second_convex_hull: list[tuple]):
        # Plotando novamente os pontos na figura
        plt.scatter(self.first_label_train_x, self.first_label_train_y, color=self.first_label_color)
        plt.scatter(self.second_label_train_x, self.second_label_train_y, color=self.second_label_color)

        self.first_convex_hull_x = [point[0] for point in first_convex_hull] + [first_convex_hull[0][0]]
        self.first_convex_hull_y = [point[1] for point in first_convex_hull] + [first_convex_hull[0][1]]
        plt.plot(self.first_convex_hull_x, self.first_convex_hull_y, color=self.first_label_color)

        self.second_convex_hull_x = [point[0] for point in second_convex_hull] + [second_convex_hull[0][0]]
        self.second_convex_hull_y = [point[1] for point in second_convex_hull] + [second_convex_hull[0][1]]
        plt.plot(self.second_convex_hull_x, self.second_convex_hull_y, color=self.second_label_color)

        plt.show()

    def plot_hull_intersection(self, first_segment: list[tuple], second_segment: list[tuple]):
        # Plotando somente as envoltórias convexas
        plt.plot(self.first_convex_hull_x, self.first_convex_hull_y, color=self.first_label_color)
        plt.plot(self.second_convex_hull_x, self.second_convex_hull_y, color=self.second_label_color)

        if first_segment is None and second_segment is None:
            plt.plot(self.first_convex_hull_x, self.first_convex_hull_y, color=self.intersection_color)
            plt.plot(self.second_convex_hull_x, self.second_convex_hull_y, color=self.intersection_color)

            plt.show()
        else:
            first_segment_x = [point[0] for point in first_segment]
            first_segment_y = [point[1] for point in first_segment]
            plt.plot(self.first_segment_x, self.first_segment_y, color=self.intersection_color)

            second_segment_x = [point[0] for point in second_segment]
            second_segment_y = [point[1] for point in second_segment]
            plt.plot(self.second_segment_x, self.second_segment_y, color=self.intersection_color)

            plt.show()

    def plot_model_line(self, closest_from_1: tuple, closest_from_2: tuple, model_slope: float, model_constant: float):
        # Plotando pontos de treino iniciais
        plt.scatter(self.first_label_train_x, self.first_label_train_y, color=self.first_label_color)
        plt.scatter(self.second_label_train_x, self.second_label_train_y, color=self.second_label_color)
        # Plotando as arestas da envoltória convexa
        plt.plot(self.first_convex_hull_x, self.first_convex_hull_y, color=self.first_label_color)
        plt.plot(self.second_convex_hull_x, self.second_convex_hull_y, color=self.second_label_color)
        # Plotando a linha que conecta os pontos mais próximos das duas envoltórias
        self.connection_line_x = [closest_from_1[0], closest_from_2[0]]
        self.connection_line_y = [closest_from_1[1], closest_from_2[1]]
        plt.plot(self.connection_line_x, self.connection_line_y, color=self.connection_color)
        # Plotando a reta que separa as duas classes
        min_x = min(self.first_label_train_x + self.second_label_train_x)
        min_y = min(self.first_label_train_y + self.second_label_train_y)

## Metrificação do classificador
Blá blá blá

# Testes do algoritmo
Dizer de onde estão surgindo as fontes de dados, como elas são tratadas e os desafios relacionados a uma classificação eficiente no algoritmo implementado

## Redutor de dimensionalidade
Falar qual foi a estratégia empregada para a redução e como ela deve ser utilizada.

## Teste 1 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 2 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 3 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 4 (3D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 5 (3D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 6 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 7 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 8 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 9 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 10 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

# Conclusão
Comentar sobre como o experimento contribuiu para o aprendizado da matéria, como o resultado dos testes foi diferente do ideal, etc...